In [19]:
## Packages

# ##Installs
# !pip install openpyxl transformers

##Imports
import pandas as pd
import transformers
import openpyxl

from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
## Reading in the PSYTar data set and parsing it
fileName=".\ONLINE_FORA\PsyTAR_dataset.xlsx"
data=pd.ExcelFile(fileName)
sheets={}
for sheet in data.sheet_names:
    sheets[sheet]=data.parse(sheet)

## Remove the first two sheets (License and read_me)
sheets.pop('License',None)
sheets.pop('read_me',None)

## This will print out the sheet names for the whole excel
for sheet in sheets.keys():
    print(f"Sheet_Name: {sheet}")

#To access a sheet, perform sheet['Sheet_Name']; e.g., sheets['Sample']

Sheet_Name: Sample
Sheet_Name: Sentence_Labeling
Sheet_Name: ADR_Identified
Sheet_Name: ADR_Mapped
Sheet_Name: WD_Identified
Sheet_Name: WD-Mapped 
Sheet_Name: SSI_Identified
Sheet_Name: SSI_Mapped
Sheet_Name: DI_Identified
Sheet_Name: DI_Mapped


In [ ]:
## Vectorize data into TF-IDF

In [16]:
## Investigate K-means clustering to see whether certain medicines have different side effects - use as baseline to see if there is a significant difference with the transformer model


In [ ]:
## Create a transformer